In [1]:
#!pip install -r requirements.txt

Looking in indexes: https://pypi.python.org/simple


In [2]:
import mne
import warnings
import numpy as np


from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import FastICA
from sklearn.neural_network import MLPClassifier

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.preprocessing import ICA

from utils.Pipeline import FourierTransform, Float64CSP, EEGStandardScaler, WaveletTransform
from utils.Preprocessing import load_and_preprocess_data


In [3]:
mne.set_log_level("CRITICAL")

# Suppress warnings from LogisticRegression and CSP
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.linear_model._logistic')
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mne')

In [4]:
# Define subjects and runs for training and testing
train_subjects = [57, 22, 86]
train_tasks = {
    'Task 1': [3, 7, 11],
    'Task 2': [4, 8, 12],
    'Task 3': [5, 9, 13],
    'Task 4': [6, 10, 14]
}

test_subjects = [34]
test_tasks = {
    'Task 1': [3],
    'Task 2': [4],
    'Task 3': [5],
    'Task 4': [6]
}

In [5]:
#task_1 (open and close left or right fist)
train_runs = train_tasks['Task 1']
train_epochs = load_and_preprocess_data(train_subjects, train_runs)

X_train = train_epochs.get_data().astype('float64')
y_train = train_epochs.events[:, -1]

display(X_train.shape)
display(y_train.shape)


test_runs = test_tasks['Task 1']
test_epochs = load_and_preprocess_data(test_subjects, test_runs)

X_test = test_epochs.get_data().astype('float64')
y_test = test_epochs.events[:, -1]

display(X_test.shape)
display(y_test.shape)

(195, 64, 481)

(195,)

(22, 64, 481)

(22,)

In [6]:
# Aplanar los datos de entrenamiento
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Visualizar las nuevas formas
display(X_train_flattened.shape)
display(X_train.shape)

display(X_test_flattened.shape)


(195, 30784)

(195, 64, 481)

(22, 30784)

-----

------

---

In [11]:
wave_mlp = Pipeline([             # Paso 2: Estandarización
    ('wavelet', WaveletTransform()),            # Paso 4: Reducción de dimensionalidad con PCA
    ('mlp', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500))  # Paso 5: Red Neuronal
])

mlp.fit(X_train, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = mlp.score(X_train, y_train)
test_accuracy = mlp.score(X_test, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(mlp, X_train, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 1.00
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.71794872 0.76923077 0.82051282 0.87179487 0.79487179]
Precisión media: 0.79


In [8]:
mlp = Pipeline([
    ('scaler', StandardScaler()),               # Paso 2: Estandarización
    #('ica', FastICA(n_components=20, random_state=42)),  # Paso 3: Análisis de Componentes Independientes
    ('pca', PCA(n_components=10)),              # Paso 4: Reducción de dimensionalidad con PCA
    ('mlp', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500))  # Paso 5: Red Neuronal
])

mlp.fit(X_train_flattened, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = mlp.score(X_train_flattened, y_train)
test_accuracy = mlp.score(X_test_flattened, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(mlp, X_train_flattened, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 1.00
Precisión en prueba: 0.73
Precisión en cada pliegue: [0.58974359 0.76923077 0.69230769 0.76923077 0.74358974]
Precisión media: 0.71


In [7]:
mlp = Pipeline([
    ('scaler', StandardScaler()),               # Paso 2: Estandarización
    #('ica', FastICA(n_components=20, random_state=42)),  # Paso 3: Análisis de Componentes Independientes
    #('pca', PCA(n_components=10)),              # Paso 4: Reducción de dimensionalidad con PCA
    ('mlp', MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500))  # Paso 5: Red Neuronal
])

mlp.fit(X_train_flattened, y_train)
# Evaluar el modelo en el conjunto de prueba
train_accuracy = mlp.score(X_train_flattened, y_train)
test_accuracy = mlp.score(X_test_flattened, y_test)

# Imprimir las precisiones
print(f"Precisión en entrenamiento: {train_accuracy:.2f}")
print(f"Precisión en prueba: {test_accuracy:.2f}")


cv_scores = cross_val_score(mlp, X_train_flattened, y_train, cv=5)

print(f"Precisión en cada pliegue: {cv_scores}")
print(f"Precisión media: {np.mean(cv_scores):.2f}")

Precisión en entrenamiento: 1.00
Precisión en prueba: 0.64
Precisión en cada pliegue: [0.74358974 0.82051282 0.82051282 0.87179487 0.79487179]
Precisión media: 0.81
